# 기본과제 - 외부 블로그의 정보와 함께 챗봇 구현하기

- [x]  RAG internet source를 https://spartacodingclub.kr/blog/all-in-challenge_winner 로 설정합니다.
    - RAG에서 활용할 source로 위의 링크를 전달합니다.
    - 사이트가 달라졌기 때문에 이전 실습 코드와 다르게 load 해야 합니다. 어디를 어떻게 수정해야 할지 고민해보도록 합시다.
    - LLM은 GPT를 사용하시면 됩니다. 모델은 `gpt-4o-mini`로 설정하시면 됩니다.
- [x]  GPT에게 `“ALL-in 코딩 공모전 수상작들을 요약해줘.”`를 물은 뒤의 답변을 출력합니다.

## 1. 필요한 라이브러리 설치 및 import

In [ ]:
!pip install langchain-community langchain-chroma langchain-openai bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.1/434.1 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00


In [ ]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
import getpass
import requests

## 2. llm 모델 정의

In [ ]:
api_key = getpass.getpass('Enter your API key: ')
llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

Enter your API key: ··········


## 3. beuatifulsoup으로 크롤링 세팅

In [ ]:
class UTF8WebBaseLoader(WebBaseLoader):
  def load(self):
    docs = []
    for url in self.web_paths:
      response = requests.get(url, **(self.requests_kwargs or {}))
      response.encoding = 'utf-8'
      soup = bs4.BeautifulSoup(response.text, 'html.parser', **(self.bs_kwargs or {}))
      text = soup.get_text(**(self.bs_get_text_kwargs or {}))
      metadata = {"source": url}
      docs.append(Document(page_content=text, metadata=metadata))
    return docs

* WEbBaseLoader를 서브클래싱해서 인코딩 적용
  * WebBaseLoader를 상속 -> requests 응답 직후 utf-8로 인코딩
* `**(self.requests_kwargs or {})` : 값이 없거나 None일 때 빈 딕셔너리 {}를 쓰겠음  
  * **딕셔너리 : 딕셔너리 안의 값들을 spread해서 씀

In [ ]:
loader = UTF8WebBaseLoader(
    web_paths=("https://spartacodingclub.kr/blog/all-in-challenge_winner",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("editedContent")
        )
    ),
    bs_get_text_kwargs=dict(
        strip=True,
        separator="\n",
    ),
)
docs = loader.load()

* web_paths
  * 타입 : `Sequence[str]` : 리스트나 튜플
  * 기본값 : 빈 튜플 `()`
  * 여러 개의 웹 사이트 주소를 사용
  * 이 URL들을 병렬로 `requests`로 가져온 뒤, b4로 파싱해서 `Document` 객체로 만든다.
* bs_kwargs
  * 타입: `Dict[str, Any]`
  * b4 생성자에 그대로 전달할 키워드 인자들을 지정함.
  * parse_only의 class_ : 웹사이트 html의 class 파악
  * kwargs = keyword arguments
* bs_get_text_kwargs
  * get_text()에 넘겨 페이지 내 모든 텍스트를 한 번에 추출할 때 어떻게 연결할지, 공백 어떻게 처리할지 정함
  * get_text() : HTML 문서 트리에서 모든 텍스트 노드를 모아서 하나의 문자열을 뽑아줌
  * `separator: str = ''` : 텍스트 조각 사이에 들어갈 문자열
  * `strip: bool = False` : 각 텍스트 조각 앞뒤 공백을 제거할 지 여부

In [ ]:
from textwrap import fill
import json

for doc in docs:
    # 1) 메타데이터 예쁘게 출력 (ensure_ascii=False로 한글 깨짐 방지)
    print("Source:")
    print(json.dumps(doc.metadata, indent=2, ensure_ascii=False))
    print()

    # 2) 본문 텍스트만 꺼내서, 원하는 폭(width)으로 재포맷
    wrapped = fill(doc.page_content, width=80)  # 80자마다 줄바꿈
    print(wrapped)

    # 3) 문서 구분선
    print("\n" + "="*80 + "\n")


Source:
{
  "source": "https://spartacodingclub.kr/blog/all-in-challenge_winner"
}

코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는
이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고
있습니다. <All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서
다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다. 🏆 대상 [Lexi
Note] 언어공부 필기 웹 서비스 서비스 제작자: 다나와(김다애, 박나경) 💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장
구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를
사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤
했죠. W와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-
원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로
연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수
있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 있습니다. 필기, 사전, 번역을 한번에 쉽고 편하게
이용할 수 있죠. 더 이상 시간 낭비 없이 효율적으로 어문학을 공부하며 학습 속도도

## 4. RAG 파이프라인 구축하기

### 텍스트 청크화 -> 벡터화 -> 저장

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=OpenAIEmbeddings(api_key=api_key)
)

* **RecursiveCharacterTextSplitter**
  * 긴 텍스트를 문단("\n\n"), 문장("\n"), 단어(" ") 단위로 재귀적으로 쪼갬
  * 지정한 `chunk_size` 이하 크기의 청크로 분할해주는 도구
  * 의미 단위가 최대한 보존 된 채로 텍스트를 나눌 수 있다..?
  * chunk_size(int): 한 청크의 최대 문자 수
  * chunk_overlap(int) : 연속된 청크 사이에 겹치게 포함할 문자 수. 문맥 연속성을 조금씩 중복시켜 놓으면 RAG 시 검색 정확도가 올라감..

* **split_documents**
  * 주어진 문서를 내부적으로 `page_content` 기준으로 자른다.
  * `chunk_size`와 `chunk_overlap`에 맞춘 새로운 청크 문서 리스트로 반환함
  * 메타데이터는 원본에서 복사되어 유지됨

* **Chroma.from_documents**
  * `documents`의 각 청크 텍스트를 `embedding` 함수로 임베딩 벡터로 변환
  * Chroma DB에 저장하여 벡터 검색이 가능하도록 벡터 저장소를 초기화
  
    => `similarity_search()` 같은 메서드로 청크를 빠르게 찾아올 수 있음

In [ ]:
print(splits)


[Document(metadata={'source': 'https://spartacodingclub.kr/blog/all-in-challenge_winner'}, page_content='코딩은 더 이상 개발자만의 영역이 아닙니다. 누구나 아이디어만 있다면 창의적인 서비스를 만들어 세상을 바꿀 수 있습니다. 스파르타코딩클럽에서는 이러한 가능성을 믿고, 누구나 코딩을 통해 자신의 아이디어를 실현하고 실제 문제를 해결하는 경험을 쌓을 수 있도록 다양한 프로그램을 마련하고 있습니다.\n<All-in> 코딩 공모전은 대학생들이 캠퍼스에서 겪은 불편함과 문제를 자신만의 아이디어로 해결해보는 대회였는데요. 이번 공모전에서 다양한 혁신적인 아이디어와 열정으로 가득한 수많은 프로젝트가 탄생했습니다. 그중 뛰어난 성과를 낸 수상작 6개를 소개합니다.\n🏆\xa0대상\n[Lexi Note] 언어공부 필기 웹 서비스\n서비스 제작자: 다나와(김다애, 박나경)\n💡W는 어문학을 전공하는 대학생입니다. 매일 새로운 단어와 문장 구조를 공부하고 있지만, 효율적으로 학습하는 것이 쉽지 않았습니다. 단어의 의미를 찾기 위해 사전을 뒤적이고, 긴 문장을 이해하려고 번역기를 사용하다 보면, 필기 노트는 어느새 뒷전으로 밀려났거든요. 사전, 번역기, 원서, 필기노트를 왔다 갔다 하다 보면 시간이 다 지나가 버리곤 했죠.\nW와 같이 어문 전공생은 문법, 어휘, 문장 구조 등 다양한 자료를 학습해야 합니다. 여러 자료를 번갈아 학습하다보니 ‘사전-번역기-원서-필기노트’ 왕복으로 학습 효율이 나지 않아 고민인 경우도 많으실 거예요. <Lexi Note>는 단어를 드래그하면 네이버 사전으로 바로 연동 돼 단어의 의미를 찾으며 동시에 필기 할 수 있어요. 이외에도 번역 버튼을 누르면 파파고 번역기가 연동돼 긴 문장도 쉽게 이해할 수 있어요. 언어 학습에 필요한 할일 목록과 스케줄 템플릿을 제공하여 효율적으로 공부할 수 있습니다. 필기, 사전, 번역을 한번에 쉽고 편하게 이용할 수 있죠. 더 이상 시간

In [ ]:
# 검색기 생성
retriever = vectorstore.as_retriever()
# 프롬프트 가져오기
prompt = hub.pull("rlm/rag-prompt")

/usr/local/lib/python3.11/dist-packages/langsmith/client.py:280: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


* `retriever` : 벡터 저장소의 검색기, 조회기
* 어떤 질문이 던져졌을 때, 이 질문과 유사한 임베딩을 가지고 있는 text를 추출함
* 질문 입력 -> 질문 임베딩으로 변환 -> vectorstore에 있는 임베딩들과 비교 -> 비슷하다고 판단되는 임베딩 반환

In [ ]:
# RAG를 통해 추출된 text들을 모두 concat함
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


## 랭체인의 chain + LCEL으로 쉽게 구현해보기

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

user_msg = "ALL-in 코딩 공모전 수상작들을 요약해줘."

# 입력 바인딩 -> 프롬프트 invoke → LLM invoke → 파싱
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

`{"context": retriever | format_docs, "question": RunnablePassthrough()}`

`Runnable` : 실행할 수 있는 무언가.. 입력을 받고 처리하고 결과를 내놓는

* 이 dict를 하나의 실행 단위(Runnable)로 본다.
* 사용자의 질문을 받음 -> "context"에 해당하는 작업 실행 -> "question"에 해당하는 작업 실행 -> 그 두 결과를 `{"context": ..., "question":...}` 형태로 묶어서 다음 단계로 넘긴다.

* `"context": retriever | format_docs`
  * `retriever` (=`vectorestore.as_retriever()`)가 먼저 호출돼서 입력 맵에 `question` 값을 꺼내 벡터 검색함
  * `format_docs`가 검색된 `Document`리스트를 포매팅
  * 파이프 (|) : 이전 Runnable 출력 -> 다음 Runnable 입력을 잇는 역할

In [ ]:
print(rag_chain.invoke(user_msg))

ALL-in 코딩 공모전의 수상작으로는 언어학습 웹서비스 'Lexi Note', 연합동아리 정보 플랫폼 'Crewing', 학교생활 관리 서비스 '학교생활 매니저', 그리고 수업 실시간 소통 서비스 '에코 클래스룸'이 있습니다. 이들은 캠퍼스 내 문제를 해결하기 위해 대학생들이 창의적으로 개발한 프로젝트로, 다양한 기술 스택을 사용하여 효율적인 서비스를 제공하고 있습니다. 공모전은 비전공자도 참여할 수 있는 기회를 제공하여 코딩과 아이디어 실현의 장이 되었습니다.
